# Example 0

This notebook will go over a few of the basics of generating new physics events with DarkNews.

We will cover:
* generating events on the command line
* generating events in python GenLauncher
* loading pandas dataframes with event information
* loading numpy arrays with event information

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import DarkNews

In [3]:
import os
import numpy as np
import pandas as pd

import DarkNews as dn
from DarkNews import const

## Generating events
Let us start by generating a few dark neutrino events for two cases of interest.

We do this in several ways. 

### `dn_gen` command line functionality

It is possible to run the generator through the script `dn_gen`, passing the parameters as options.
The created dataset can be found in the `data/` directory tree, which is created in the same folder the script is run.

Let's try to run few points.

In [29]:
cmd_string = "dn_gen --mzprime=1.25 --m4=0.140 --neval=1000 --HNLtype=dirac --loglevel=INFO --hepevt"
const.subprocess_cmd(cmd_string)

dn_gen --mzprime=1.25 --m4=0.140 --neval=1000 --HNLtype=dirac --loglevel=INFO --hepevt

/bin/sh: dn_gen: command not found



Load the dataset.

In [30]:
df_d = pd.read_pickle('./data/miniboone_fhc/3plus1/m4_0.14_mzprime_1.25_dirac/pandas_df.pckl')
list(df_d.columns.levels[0])

['P_decay_N_daughter',
 'P_decay_N_parent',
 'P_decay_ell_minus',
 'P_decay_ell_plus',
 'P_projectile',
 'P_recoil',
 'P_target',
 'pos_scatt',
 'w_decay_rate_0',
 'w_event_rate',
 'w_flux_avg_xsec',
 'target',
 'target_pdgid',
 'scattering_regime',
 'helicity',
 'underlying_process',
 'pos_decay']

### `GenLauncher` class
It is possible to run the generator through an instance of the `GenLauncher` class in the `DarkNews.GenLauncher` module.
We create an instance of this class, specifying the parameters of the run, then we use the `run()` method on this instance.
The created dataset can be found in the `data/` directory tree.
Alternatively, it an be accessed via the `df` attribute.

We run the same points as before, but with the `GenLauncher` class. When using the `run()` method it is possible to specify some parameters as `log`, `verbose` or `logfile` (which can be also specified during the construction of the object).

In [28]:
from DarkNews.GenLauncher import GenLauncher
gen_object = GenLauncher(mzprime=1.25, m4=0.140, neval=1000, noHF=True, HNLtype="dirac")
df_d = gen_object.run()

---------------------------------------------------------
   ______           _        _   _                     
   |  _  \         | |      | \ | |                    
   | | | |__ _ _ __| | __   |  \| | _____      _____   
   | | | / _  | ___| |/ /   | .   |/ _ \ \ /\ / / __|  
   | |/ / (_| | |  |   <    | |\  |  __/\ V  V /\__ \  
   |___/ \__,_|_|  |_|\_\   \_| \_/\___| \_/\_/ |___/  

---------------------------------------------------------
Model:
	1 dirac heavy neutrino(s).
	kinetically mixed Z'

---------------------------------------------------------
Experiment:
	MiniBooNE_FHC
	fluxfile loaded: ../fluxes/MiniBooNE_FHC.dat
	POT: 1.875e+21
	nuclear targets: ['C12', 'H1']
	fiducial mass: [701.1428571428571, 116.85714285714286] tonnes

Note that the directory tree for this run already exists.
---------------------------------------------------------
Generating Events using the neutrino-nucleus upscattering engine

nu(mu) C12 --> N4  C12 --> nu_light e+ e- C12
Helicity conservin

In [25]:
gen_object = GenLauncher(Umu5=1e-3, UD5=35, chi=0.0031, gD=2, mzprime=1.25, m4=0.080, m5=0.140, neval=1000, HNLtype="majorana")
df_m=gen_object.run(loglevel="INFO")

NameError: name 'GenLauncher' is not defined

### File interface
It is possible to specify the parameters through a file.
This can be done either via the command-line inerface, thorugh the option `--param-file` or via the `GenLauncher` class, thorugh the argument `param-file` in the constructor.
In any case, the parameters specified in the file will oeverwrite the default ones, but will be overwritten by any other definition (either via options in command-line mode or via the keyword arguments in the `GenLauncher` constructor) of the same variable.

A template file [`template_parameters_file.txt`](examples/template_parameters_file.txt) can be found in this directory.
The file interface allows the user to specify the parameters with far more freedom, enabling the possibility to use mathematical expressions, involving user-defined constants, while keeping all the parameters in one single file.

In [14]:
cmd_string = "dn_gen --param-file=./parameters_example_1.txt"
const.subprocess_cmd(cmd_string)

dn_gen --param-file=./parameters_example_1.txt

/bin/sh: dn_gen: command not found



In [20]:
from DarkNews.GenLauncher import GenLauncher
gen_object = GenLauncher(param_file="./parameters_example_1.txt")
df_1 = gen_object.run(loglevel="INFO")

AttributeError: Parameters hbar, c were unused. Either not supported or spelled wrong.

In [22]:
cmd_string = "dn_gen --param-file=./parameters_example_2.txt"
const.subprocess_cmd(cmd_string)

dn_gen --param-file=./parameters_example_2.txt

/bin/sh: dn_gen: command not found



In [24]:
gen_object = GenLauncher(param_file="./parameters_example_2.txt")
df_2=gen_object.run(loglevel="INFO")

AttributeError: Parameters hbar, c, umu5, ud5 were unused. Either not supported or spelled wrong.

### Select an experiment

It is possible to select an experiment through the `exp` argument of the `GenLauncher` class (or `--exp` option of the command line interface):
1. specifying a keyword for a pre-defined experiment among:
    * DUNE FHC ND (`"dune_nd_fhc"`)
    * DUNE RHC ND (`"dune_nd_rhc"`)
    * MicroBooNE (`"microboone"`)
    * MINERVA FHC LE (`"minerva_le_fhc"`)
    * MINERVA FHC ME (`"minerva_me_fhc"`)
    * MINERVA RHC ME (`"minerva_me_rhc"`)
    * MiniBooNE FHC (`"miniboone_fhc"`)
    * MINOS FHC ME (`"minos_le_fhc"`)
    * MINOS FHC LE (`"minos_me_fhc"`)
    * ND280 FHC (`"nd280_fhc"`)
    * NOva FHC (`"nova_le_fhc"`)
    * NuTeV FHC (`"nutev_fhc"`)
    * FASERv (`"fasernu"`)
1. specifying the file path of an experiment file: every file should be specified using the same rules as for the parameters file.
A template file [`template_custom_experiment.txt`](examples/template_custom_experiment.txt) can be found in this directory.

In [25]:
cmd_string = "dn_gen --exp=microboone"
const.subprocess_cmd(cmd_string)

dn_gen --exp=microboone

/bin/sh: dn_gen: command not found



In [26]:
cmd_string = "dn_gen --exp=./template_custom_experiment.txt"
const.subprocess_cmd(cmd_string)

dn_gen --exp=./template_custom_experiment.txt

/bin/sh: dn_gen: command not found



In [27]:
from DarkNews.GenLauncher import GenLauncher
gen_object = GenLauncher(exp="microboone")
gen_object.run(loglevel="INFO")

---------------------------------------------------------
   ______           _        _   _                     
   |  _  \         | |      | \ | |                    
   | | | |__ _ _ __| | __   |  \| | _____      _____   
   | | | / _  | ___| |/ /   | .   |/ _ \ \ /\ / / __|  
   | |/ / (_| | |  |   <    | |\  |  __/\ V  V /\__ \  
   |___/ \__,_|_|  |_|\_\   \_| \_/\___| \_/\_/ |___/  

---------------------------------------------------------
Model:
	1 majorana heavy neutrino(s).
	kinetically mixed Z'

---------------------------------------------------------
Experiment:
	MicroBooNE
	fluxfile loaded: ../fluxes/MiniBooNE_FHC.dat
	POT: 1.225e+21
	nuclear targets: ['Ar40']
	fiducial mass: [85.0] tonnes

---------------------------------------------------------
Generating Events using the neutrino-nucleus upscattering engine

nu(mu) Ar40 --> N4  Ar40 --> nu_light e+ e- Ar40
Helicity conserving upscattering.
N4 decays via off-shell Z'.
Predicted (792 +/- 9.48) events.

nu(mu) Ar40 -->

P_projectile                       P_target                          \
                 0    1    2         3          0    1    2             3   
0         0.309515  0.0  0.0  0.309515  37.215541 -0.0 -0.0  4.761747e-13   
1         0.173824  0.0  0.0  0.173824  37.215541 -0.0 -0.0  4.742595e-13   
2         0.424319  0.0  0.0  0.424319  37.215541 -0.0 -0.0  2.235434e-13   
3         0.296294  0.0  0.0  0.296294  37.215541 -0.0 -0.0 -2.564615e-13   
4         0.190697  0.0  0.0  0.190697  37.215541 -0.0 -0.0  7.855105e-13   
...            ...  ...  ...       ...        ...  ...  ...           ...   
35814     1.319565  0.0  0.0  1.319565   0.938272 -0.0 -0.0  1.110223e-16   
35815     2.009923  0.0  0.0  2.009923   0.938272 -0.0 -0.0  2.220446e-16   
35816     1.374438  0.0  0.0  1.374438   0.938272 -0.0 -0.0 -1.110223e-16   
35817     1.510476  0.0  0.0  1.510476   0.938272 -0.0 -0.0 -2.220446e-16   
35818     2.249223  0.0  0.0  2.249223   0.938272 -0.0 -0.0  0.000000e+00   

        P_recoil            ... w_flux_avg_xsec          target target_pdgid  \
               0         1  ...                                                
0      37.215575  0.008179  ...        0.143982            Ar40   1000180400   
1      37.215668 -0.006532  ...        0.095384            Ar40   1000180400   
2      37.215580 -0.040893  ...        0.118122            Ar40   1000180400   
3      37.215621  0.031852  ...        0.099263            Ar40   1000180400   
4      37.215638 -0.047499  ...        0.140265            Ar40   1000180400   
...          ...       ...  ...             ...             ...          ...   
35814   1.207128 -0.377994  ...        0.060828  proton_in_Ar40         2212   
35815   1.500631  0.819828  ...        0.029585  proton_in_Ar40         2212   
35816   1.927307  0.018218  ...        0.013866  proton_in_Ar40         2212   
35817   1.281906  0.498418  ...        0.050895  proton_in_Ar40         2212   
35818   2.188861 -0.024750  ...        0.006785  proton_in_Ar40         2212   

      scattering_regime    helicity  \
                                      
0              coherent  conserving   
1              coherent  conserving   
2              coherent  conserving   
3              coherent  conserving   
4              coherent  conserving   
...                 ...         ...   
35814              p-el    flipping   
35815              p-el    flipping   
35816              p-el    flipping   
35817              p-el    flipping   
35818              p-el    flipping   

                                      underlying_process    pos_decay  \
                                                                    0   
0       nu(mu) Ar40 --> N4  Ar40 --> nu_light e+ e- Ar40  4632.313430   
1       nu(mu) Ar40 --> N4  Ar40 --> nu_light e+ e- Ar40  4523.134181   
2       nu(mu) Ar40 --> N4  Ar40 --> nu_light e+ e- Ar40  4493.093781   
3       nu(mu) Ar40 --> N4  Ar40 --> nu_light e+ e- Ar40  3716.235942   
4       nu(mu) Ar40 --> N4  Ar40 --> nu_light e+ e- Ar40  4554.892813   
...                                                  ...          ...   
35814  nu(mu) proton_in_Ar40 --> N4  proton_in_Ar40 -...  3565.788391   
35815  nu(mu) proton_in_Ar40 --> N4  proton_in_Ar40 -...  3590.012530   
35816  nu(mu) proton_in_Ar40 --> N4  proton_in_Ar40 -...  4043.474611   
35817  nu(mu) proton_in_Ar40 --> N4  proton_in_Ar40 -...  3503.459435   
35818  nu(mu) proton_in_Ar40 --> N4  proton_in_Ar40 -...  4388.672324   

                                           
                1           2           3  
0        6.154774   65.694957  475.415649  
1     -151.456720  -30.415818  494.303776  
2       54.807253 -159.484851  -95.994545  
3        0.629934   20.676286  510.053585  
4       10.732870  156.768461  -51.827636  
...           ...         ...         ...  
35814  145.359704   -4.342245 -301.318206  
35815 -152.276136   99.796354  460.586551  
35816  -65.102059   57.222049  440.523814  
35817 -136.820303    

In [28]:
gen_object = GenLauncher(exp="./template_custom_experiment.txt")
gen_object.run(loglevel="INFO")

---------------------------------------------------------
   ______           _        _   _                     
   |  _  \         | |      | \ | |                    
   | | | |__ _ _ __| | __   |  \| | _____      _____   
   | | | / _  | ___| |/ /   | .   |/ _ \ \ /\ / / __|  
   | |/ / (_| | |  |   <    | |\  |  __/\ V  V /\__ \  
   |___/ \__,_|_|  |_|\_\   \_| \_/\___| \_/\_/ |___/  

---------------------------------------------------------
Model:
	1 majorana heavy neutrino(s).
	kinetically mixed Z'

---------------------------------------------------------
Experiment:
	My Experiment
	fluxfile loaded: ./fluxfile_uniform_0.1_200_GeV.dat
	POT: 1e+20
	nuclear targets: ['H1', 'He3', 'Li3']
	fiducial mass: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333] tonnes

---------------------------------------------------------
Generating Events using the neutrino-nucleus upscattering engine

nu(mu) proton_in_H1 --> N4  proton_in_H1 --> nu_light e+ e- proton_in_H1
Helicity conser

P_projectile                       P_target                          \
                 0    1    2          3         0    1    2             3   
0         3.424048  0.0  0.0   3.424048  0.938272 -0.0 -0.0  0.000000e+00   
1         3.553188  0.0  0.0   3.553188  0.938272 -0.0 -0.0 -2.220446e-16   
2         4.622415  0.0  0.0   4.622415  0.938272 -0.0 -0.0 -4.440892e-16   
3         4.967546  0.0  0.0   4.967546  0.938272 -0.0 -0.0  0.000000e+00   
4         2.152411  0.0  0.0   2.152411  0.938272 -0.0 -0.0 -6.661338e-16   
...            ...  ...  ...        ...       ...  ...  ...           ...   
89607    17.400897  0.0  0.0  17.400897  0.938272 -0.0 -0.0  1.776357e-15   
89608    17.712324  0.0  0.0  17.712324  0.938272 -0.0 -0.0  3.552714e-15   
89609    16.448946  0.0  0.0  16.448946  0.938272 -0.0 -0.0  0.000000e+00   
89610    16.454606  0.0  0.0  16.454606  0.938272 -0.0 -0.0 -1.776357e-15   
89611    16.831611  0.0  0.0  16.831611  0.938272 -0.0 -0.0  1.776357e-15   

       P_recoil            ... w_flux_avg_xsec         target target_pdgid  \
              0         1  ...                                               
0      0.951883  0.158272  ...    3.259003e+06   proton_in_H1         2212   
1      0.945161 -0.086756  ...    3.551220e+06   proton_in_H1         2212   
2      0.970573  0.244896  ...    3.089254e+06   proton_in_H1         2212   
3      0.945038 -0.026397  ...    3.723609e+06   proton_in_H1         2212   
4      0.949107 -0.065392  ...    3.280764e+06   proton_in_H1         2212   
...         ...       ...  ...             ...            ...          ...   
89607  2.210600 -0.844876  ...    6.153554e+05  proton_in_Li3         2212   
89608  1.639102  1.122900  ...    1.586001e+06  proton_in_Li3         2212   
89609  1.195541  0.183650  ...    2.831330e+06  proton_in_Li3         2212   
89610  1.425965  0.940146  ...    2.188321e+06  proton_in_Li3         2212   
89611  1.855799  0.915553  ...    1.103359e+06  proton_in_Li3         2212   

      scattering_regime    helicity  \
                                      
0                  p-el  conserving   
1                  p-el  conserving   
2                  p-el  conserving   
3                  p-el  conserving   
4                  p-el  conserving   
...                 ...         ...   
89607              p-el    flipping   
89608              p-el    flipping   
89609              p-el    flipping   
89610              p-el    flipping   
89611              p-el    flipping   

                                      underlying_process pos_decay            \
                                                                 0    1    2   
0      nu(mu) proton_in_H1 --> N4  proton_in_H1 --> n...       0.0  0.0  0.0   
1      nu(mu) proton_in_H1 --> N4  proton_in_H1 --> n...       0.0  0.0  0.0   
2      nu(mu) proton_in_H1 --> N4  proton_in_H1 --> n...       0.0  0.0  0.0   
3      nu(mu) proton_in_H1 --> N4  proton_in_H1 --> n...       0.0  0.0  0.0   
4      nu(mu) proton_in_H1 --> N4  proton_in_H1 --> n...       0.0  0.0  0.0   
...                                                  ...       ...  ...  ...   
89607  nu(mu) proton_in_Li3 --> N4  proton_in_Li3 -->...       0.0  0.0  0.0   
89608  nu(mu) proton_in_Li3 --> N4  proton_in_Li3 -->...       0.0  0.0  0.0   
89609  nu(mu) proton_in_Li3 --> N4  proton_in_Li3 -->...       0.0  0.0  0.0   
89610  nu(mu) proton_in_Li3 --> N4  proton_in_Li3 -->...       0.0  0.0  0.0   
89611  nu(mu) proton_in_Li3 --> N4  proton_in_Li3 -->...       0.0  0.0  0.0   

            
         3  
0      0.0  
1      0.0  
2      0.0  
3      0.0  
4      0.0  
...    ...  
89607  0.0  
89608  0.0  
89609  0.0  
89610  0.0  
89611  0.0  

[89612 rows x 44 columns]

---
# Pandas DataFrame Output

### Metadata

Before inspecting the events inside the pandas dataframe, let us check the metadata in [df.attrs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.attrs.html). We have:

---
1. **experiment**: this is a class DarkNews.experiment.Detector, which contains all the information about the experiment for which the events are generated. It also contains a list of instances of the NuclearTarget class, which contains information about the different scattering target used.

In [ ]:
print("---------------------------------------------")
print("General description of the detector attribute")
print("---------------------------------------------")
keys = list(df_d.attrs['experiment'].__dict__.keys())
print(f"Attributes of Detector class: \n", *keys)
print(f"\nAttributes of NuclearTarget class: \n", *list(df_d.attrs['experiment'].NUCLEAR_TARGETS[0].__dict__))

In [ ]:
print("--------------------------------------------")
print("How much of each target inside the detector?")
print("--------------------------------------------")
print("\nExample of how to look up nuclear targets:")
fid_mass = df_d.attrs['experiment'].FIDUCIAL_MASS
for target, fid_mass_fraction in zip(df_d.attrs['experiment'].NUCLEAR_TARGETS, df_d.attrs['experiment'].FIDUCIAL_MASS_FRACTION_PER_TARGET):
    print("{} tonnes of {}".format(round(fid_mass_fraction*fid_mass,2), target.NAME))

---
2. **bsm_model**: this attribute contains DarkNews.model.Model, which stores all the new physics parameters used in the generation. This includes both low-level information, such as the couplings, say $g_D$, as well as high-level ones, such as the couplings of the $Z^\prime$ to electrons, $d_e^V$ and $d_e^A$.

In [ ]:
print("Some low-level parameters:\n", *list(df_d.attrs['bsm_model'].__dict__.keys())[:19])
print("\nSome high-level parameters:\n", *list(df_d.attrs['bsm_model'].__dict__.keys())[-14:])

---
### Description of the event structure
Now we look at the actual events. The DataFrame is a MultiIndex df. Each event contains the components for all the 4-momenta of the particles involved:

$$\nu _\text{P_projectile} \,+\, \text{Hadronic target} _\text{P_target} \to N _\text{P_decay_N_parent} \,+\, \text{Hadronic recoil} _\text{P_recoil}$$

$$N _\text{P_decay_N_parent} \to N^\prime _\text{P_decay_N_daughter} \,+\, \ell^+ _\text{P_decay_ell_plus} \,+\, \ell^-_\text{P_decay_ell_minus}$$

In [ ]:
df_d.dtypes

##

* w_decay_rate_0           float64 -- weight of the decay rate (sum(w) = Gamma_N)
* I_decay_rate_0           float64 -- total rate Gamma_N


* w_event_rate             float64 -- weight for the event rate (sum(w) = event rate)
* I_event_rate             float64 -- total event rate


* w_flux_avg_xsec          float64 -- weight of the flux averaged cross section (sum(w) = int(sigma*flux)* exposure)
* I_flux_avg_xsec          float64 -- int(sigma*flux)* exposure


* target                    object -- target object -- it will typically be a nucleus
* target_pdgid               int64 -- pdgID of the target 


* scattering_regime         object -- regime can be coherent or p-elastic
* helicity                  object -- helicity process, can be flipping or conserving. flipping is suppressed
* underlying_process        object -- string of the underlying process, e.g, "nu(mu) + proton_in_C12 -> N4 +  proton_in_C12 -> nu(mu) + e+ + e- + proton_in_C12"

In [ ]:
np.shape(df_d['P_decay_N_parent'].to_numpy())

In [ ]:
from DarkNews import fourvec as fv

In [ ]:
fv.df_dot4(df_d['P_decay_N_parent'],df_d['P_decay_N_parent'])